# Introduction

---

This notebook details the complete workflow for building and evaluating a multi-label image classification model using the EfficientNet B0 architecture. The goal is to predict multiple labels simultaneously for each image within a dataset. This notebook uses PyTorch for model building and training, scikit-learn for data splitting and some evaluation metrics. The process is divided into several key steps, outlined below.

# Import Libraries

In [2]:
import torchvision
from torch.utils.data import Dataset, DataLoader
import os
import pandas as pd
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
from torchvision import datasets, transforms
from tqdm.auto import tqdm
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve, auc, roc_auc_score
import seaborn as sns
import numpy as np
from torchvision import models, transforms
import torch.nn.functional as F
from torch.utils.tensorboard import SummaryWriter
import itertools
from sklearn.metrics import classification_report
import random
import json
from plots import *
from helper import *
from metrics import *

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [4]:
#be fruitful and multiply
seed = 42
np.random.seed(seed)
random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

# Load Files

---
This step loads the image data and corresponding labels. The image paths are read from a CSV file ('annotations.csv'), which contains filenames and their associated labels with associated feature presence.  The images the dataset used for this project is a collection of images of beehive frames with varying features. The images themselves are loaded from the 'images/raw/' directory.


In [6]:
data_dir = 'data'
csv_path = os.path.join(data_dir, 'annotations.csv')
img_path = os.path.join(data_dir, 'images')
df = pd.read_csv(csv_path)

# Model Definition and Training

---

A custom dataset class ('CustomImageDataset') is defined to efficiently handle the image and label data during training. The pre-trained EfficientNet-B0 model is loaded, and its final classification layer is adjusted to match the number of output labels (15 in this case). The model is then trained using binary cross-entropy loss ('BCEWithLogitsLoss') and an Adam optimizer. The training process is monitored and logged, with metrics recorded at each epoch.

In [8]:
class CustomImageDataset(Dataset):
    def __init__(self, images, labels, data_transform=None):
        self.images = images
        self.labels = labels
        self.transform = data_transform
    def __len__(self):
        return len(self.images)
    def __getitem__(self, idx):
        image = self.images[idx]
        label = self.labels[idx]
        if self.transform:
            image = self.transform(image)
        label = torch.tensor(label, dtype=torch.float32)
        sample = {'Image': image, 'Label': label}
        return sample

In [1]:
# Read in label file (annotations.csv)
df = pd.read_csv(csv_path)
df = df[:5]

# Convert all labels to floats, NaN if blank
for col in df.columns[1:16]:
    df[col] = pd.to_numeric(df[col], errors='coerce')

# Drop any images with NaNs in label columns
df.dropna(subset=df.columns[:16], inplace=True)
df.reset_index(drop=True, inplace=True)

# Extract ground truth labels for each image in label file (annotations.csv)
labels = []
for index, row in df.iterrows():
    labels.append(list(row.iloc[1:16]))

# Create path for and load in each image in directory
image_names = df['filename'].tolist()
images = []

for img_name in tqdm(image_names):
    image_path = os.path.join(img_path, img_name)
    image = Image.open(image_path)
    if image.mode != 'RGB':
        image = image.convert('RGB')
    images.append(image)

# Split data into train (80%), validation (10%), and test (10%) sets
X_train_val, X_test, y_train_val, y_test = train_test_split(images, labels, test_size=1/10, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=1/9, random_state=42)

In [2]:
feature_distribution_train = pd.DataFrame(np.array(y_train).sum(axis=0), columns=['count'], index=df.columns[1:16])
at_the_bar(feature_distribution_train['count'], 'Feature Distribution - Training Set', 'feature_distribution_train.png', 'eb0', 'train', save_dir)

In [3]:
feature_distribution_val = pd.DataFrame(np.array(y_val).sum(axis=0), columns=['count'], index=df.columns[1:16])
at_the_bar(feature_distribution_val['count'], 'Feature Distribution - Validation Set', 'feature_distribution_val.png', 'eb0', 'val', save_dir)

In [4]:
feature_distribution_test = pd.DataFrame(np.array(y_test).sum(axis=0), columns=['count'], index=df.columns[1:16])
at_the_bar(feature_distribution_test['count'], 'Feature Distribution - Test Set', 'feature_distribution_test.png', 'eb0', 'test', save_dir)

In [14]:
# Define image transformations
eb0_train_datatransform = transforms.Compose([
    transforms.Resize(size=(224, 224)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

eb0_test_datatransform = transforms.Compose([
    transforms.Resize(size=(224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

efficientnet_b0_triv_aug_trans = transforms.Compose([
    transforms.Resize(size=(256, 256)),
    transforms.TrivialAugmentWide(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])


# Create datasets

val_data = CustomImageDataset(X_val, y_val, data_transform=eb0_test_datatransform)
test_data = CustomImageDataset(X_test, y_test, data_transform=eb0_test_datatransform)

# Create data loaders
BATCH_SIZE = 32

val_dataloader = DataLoader(val_data, batch_size=BATCH_SIZE, shuffle=False)
test_dataloader = DataLoader(test_data, batch_size=BATCH_SIZE, shuffle=False)


# --- EfficientNet-B0 Model Definition ---
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

weights = models.EfficientNet_B0_Weights.DEFAULT
model_eb0 = models.efficientnet_b0(weights=weights).to(device)

# Freeze pre-trained layers (optional)
for param in model_eb0.parameters():
    param.requires_grad = False

# Modify the final classifier layer
input_shape = model_eb0.classifier[1].in_features
output_shape = 15
model_eb0.classifier = nn.Sequential(
    nn.Dropout(p=0.2, inplace=True),
    nn.Linear(input_shape, output_shape)
).to(device)


# --- Training and Evaluation ---
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model_eb0.parameters(), lr=0.001)



#Hyperparameter Tuning

---
To optimize performance, a hyperparameter search is conducted. Differnt combinations of batch size, number of epochs, and learning rate are tested, using both the original data transformations and a new transformation using 'transforms.TriviaAugmentWide'. The results of this search are saved to 'hyperparameter_testing_EfficientNet_B0.csv' for analysis. The best performing hyperparameter set is then saved to 'best_params_eb0'.


In [16]:
num_epochs = [5, 10]
lrs = [.001, .0001, .01]
batch_size = [16, 32, 64]

hyperparameter_combos = list(itertools.product(batch_size, num_epochs, lrs))
results = []

In [5]:
#--------------------------------------Hyperparameter Testing--------------------------------------------------
# Create two datasets, one for each transform
train_data_triv_aug = CustomImageDataset(X_train, y_train, data_transform=efficientnet_b0_triv_aug_trans)
train_data_eb0_transform = CustomImageDataset(X_train, y_train, data_transform=eb0_train_datatransform)

for transform_name, train_data in [("triv_aug", train_data_triv_aug), ("eb0_transform", train_data_eb0_transform)]:
    for batch_size, epoch, lr in hyperparameter_combos:
        # Create DataLoader based on the current transform
        train_dataloader = DataLoader(dataset=train_data, batch_size=batch_size, shuffle=True)
        save_dir = f"runs/{transform_name}/lr_{lr}_batch_{batch_size}_epochs_{epoch}"
        os.makedirs(save_dir, exist_ok=True)
        writer = SummaryWriter(log_dir=save_dir)

        params = {'learning_rate': lr, 'batch_size': batch_size, 'epochs': epoch, 'transform': transform_name}
        writer.add_hparams(params, {})

        print(f"Starting training with hyperparameters: lr={lr}, batch_size={batch_size}, epochs={epoch}, transform={transform_name}")

        try:
            train_results = train(model=model_eb0,
                                  train_dataloader=train_dataloader,
                                  test_dataloader=val_dataloader,
                                  optimizer=torch.optim.Adam(model_eb0.parameters(), lr=lr),
                                  loss_fn=criterion,
                                  epochs=epoch,
                                  device=device,
                                  writer=writer,
                                  params=params)

            final_test_loss = train_results['test_loss'][-1]
            final_test_ham_acc = train_results['test_ham_acc'][-1]
            final_test_zero_one_acc = train_results['test_zero_one_acc'][-1]
            writer.add_hparams(params, {'Final Test Loss': final_test_loss, 'Final Test Hamming Acc': final_test_ham_acc, 'Final Test Zero-One Acc': final_test_zero_one_acc})

            results.append({'transform': transform_name, 'batch_size': batch_size, 'epochs': epoch, 'lr': lr, 'final_test_loss': final_test_loss, 'final_test_ham_acc': final_test_ham_acc, 'final_test_zero_one_acc': final_test_zero_one_acc})

        except Exception as e:
            print(f"An error occurred during training: {e}")
            results.append({'transform': transform_name, 'batch_size': batch_size, 'epochs': epoch, 'lr': lr, 'final_test_loss': float('nan'), 'final_test_ham_acc': float('nan'), 'final_test_zero_one_acc': float('nan')})

        finally:
            writer.close()

results_df = pd.DataFrame(results)
print(results_df)

In [18]:
#save result images
save_dir = os.getcwd()

In [20]:
results_df = pd.DataFrame(results)
results_df['final_test_zero_one_acc'] = results_df['final_test_zero_one_acc'].apply(lambda x: x.item())
file_path = os.path.join(save_dir, "hyperparameter_testing_d201.csv")
results_df.to_csv(file_path, index=False)

In [6]:
best_row = results_df.loc[results_df['final_test_ham_acc'].idxmax()]

best_params = {
    'transform': best_row['transform'],
    'batch_size': best_row['batch_size'].item(),
    'epochs': best_row['epochs'].item(),
    'lr': best_row['lr'].item(),
    'final_test_loss': best_row['final_test_loss'].item(),
    'final_test_ham_acc': best_row['final_test_ham_acc'].item(),
    'final_test_zero_one_acc': best_row['final_test_zero_one_acc'].item()
}

filepath = 'best_params_eb0.json'
with open(filepath, 'w') as f:
    json.dump(best_params, f, indent=4)
print(f"Best parameters saved to {filepath}")
print(best_params)

In [7]:
#train with best hyperparameters
optimizer = torch.optim.Adam(model_eb0.parameters(), lr=best_params['lr'])
train_results = train(model=model_eb0,
                      train_dataloader=train_dataloader,
                      test_dataloader=val_dataloader,
                      optimizer=optimizer,
                      loss_fn=criterion,
                      epochs=best_params['epochs'],
                      device=device)

In [8]:
test_loss, test_ham_acc, test_zero_one_acc = test_step(model=model_eb0,
                                                        dataloader=test_dataloader,
                                                        loss_fn=criterion,
                                                        device=device)

test_loss = test_loss.item() if isinstance(test_loss, torch.Tensor) else test_loss
test_ham_acc = test_ham_acc.item() if isinstance(test_ham_acc, torch.Tensor) else test_ham_acc
test_zero_one_acc = test_zero_one_acc.item() if isinstance(test_zero_one_acc, torch.Tensor) else test_zero_one_acc

test_results = {
    'Test Loss': test_loss,
    'Test Hamming Accuracy': test_ham_acc,
    'Test Zero-One Accuracy': test_zero_one_acc
}

json_filepath = os.path.join(save_dir, 'test_results_eb0.json')
with open(json_filepath, 'w') as jsonfile:
    json.dump(test_results, jsonfile, indent=4)

print(f"Test results saved to {json_filepath}")
print(f"Test Loss: {test_loss:.4f}, Test Hamming Accuracy: {test_ham_acc:.4f}, Test Zero-One Accuracy: {test_zero_one_acc:.4f}")

# Evaluations

---
After training, the model is evaluated on a held-out test set using several metrics. These include:
*   **Accuracy:** Overall correctness of predictions.
*   **Precision:** Proportion of correctly predicted positive labels among all predicted positive labels.
*   **Recall:** Proportion of correctly predicted positive labels among all actual positive labels.
*   **F1-Score:** Harmonic mean of precision and recall
*   **Confusion Matrix:** Visual represnetation of model predictions versus ground truth.
*   **ROC AUC:** Area under the ROC curve, measuring the model's ability to distinguish between classes.


In [27]:
def how_did_i_do(model, test_dataloader, device):
    """Evaluates the model and computes metrics."""
    model.eval()
    y_true = []
    y_prob = []

    with torch.no_grad():
        for batch in test_dataloader:
            inputs = batch['Image'].to(device)
            labels = batch['Label'].to(device)
            outputs = model(inputs)

            y_true.extend(labels.cpu().numpy())
            probabilities = torch.sigmoid(outputs)
            y_prob.extend(probabilities.cpu().numpy())

    y_true = np.array(y_true)
    y_prob = np.array(y_prob)

    y_pred = (y_prob >= 0.5).astype(int)

    accuracy = compute_accuracy(y_true, y_pred)
    precision = compute_precision(y_true, y_pred, average='micro')
    recall = compute_recall(y_true, y_pred, average='micro')
    f1 = compute_f1_score(y_true, y_pred, average='micro')
    confusion_mat = compute_confusion_matrix(y_true.flatten(), y_pred.flatten())
    roc_auc = compute_roc_auc(y_true.flatten(), y_prob.flatten())
    fpr, tpr, roc_auc_value = compute_roc_curve_data(y_true.flatten(), y_prob.flatten())



    # Print the metrics
    print(f"Accuracy: {accuracy}")
    print(f"Precision: {precision}")
    print(f"Recall: {recall}")
    print(f"F1-Score: {f1}")
    print(f"Confusion Matrix:\n{confusion_mat}")
    print(f"ROC AUC: {roc_auc}")
    print(f"FPR: {fpr}")
    print(f"TPR: {tpr}")
    print(f"ROC AUC Value: {roc_auc_value}")

    return accuracy, precision, recall, f1, confusion_mat, roc_auc, fpr, tpr, roc_auc_value, y_true, y_pred

In [9]:
accuracy, precision, recall, f1, confusion_mat, roc_auc, fpr, tpr, roc_auc_value, y_true, y_pred = how_did_i_do(model_eb0, test_dataloader, device)

# Visualizations

---

Finally, visualizations are generated to aid in the understanding of the model's performance. A confusion matrix is plotted to show the distribution of correct and incorre t predictions. Plots comparing training and validation loss and accuracy over epochs are generated as well as visualizations for the AUC.

## Confusion Matrix

In [10]:
plot_confusion_matrix(y_true, y_pred, df, 'eb0', save_dir)

In [11]:
plot_single_confusion_matrix(model_eb0, test_dataloader, device, 'eb0', save_dir)

##Training and Validation Comparisons per Metric

In [12]:
plot_training_validation_metrics(save_dir, train_results, 'eb0')

## Area Under the Curve (AUC)

In [13]:
plot_roc_curves(save_dir, model_eb0, test_dataloader, device, 'eb0')

# References

---

1.   **Gallery of transformations:**<br>
      PyTorch.org. "Plot Transforms Illustrations." PyTorch, https://pytorch.org/vision/stable/auto_examples/transforms/plot_transforms_illustrations.html#sphx-glr-auto-examples-transforms-plot-transforms-illustrations-py. Accessed 01 November 2024.
2.   **DataLoaders:**<br>
      PyTorch.org. "torch.utils.data.DataLoader — PyTorch 2.1.0+cu118 documentation." PyTorch, https://pytorch.org/docs/stable/data.html#torch.utils.data.DataLoader. Accessed 03 November 2024.
3.    **PyTorch Deep Learning (Train and Test Loops):**<br>
      Bourke, Daniel. "engine.py." pytorch-deep-learning, https://github.com/mrdbourke/pytorch-deep-learning/blob/main/going_modular/going_modular/engine.py. Accessed 03 November 2024.
4.    **PyTorch Torchvision Blog Post:**<br>
      PyTorch Blog. "How to Train State-of-the-Art Models Using Torchvision’s Latest Primitives." PyTorch, https://pytorch.org/blog/how-to-train-state-of-the-art-models-using-torchvision-latest-primitives/. Accessed 03 November 2024.
5.    **EfficientNet-B0 Tutorial:**<br>
      DebuggerCafe. "Transfer Learning Using EfficientNet PyTorch." DebuggerCafe, https://debuggercafe.com/transfer-learning-using-efficientnet-pytorch/. Accessed 14 November 2024.